In [1]:
from langchain.chat_models import ChatOpenAI

from langchain.memory import ConversationSummaryBufferMemory

In [2]:
# langchain에는 5가지 종류의 메모리가 있음
# 챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없음
# 유저가 자신의 이름을 말하거나 이전 질문에 이어지는 질문을 해도 기억을 못하기 때문에 대화를 이해할 수 없음

# openai에서 제공하는 기본 api는 langchain없이 사용할 수 있는데
# 메모리를 지원하지 않음 -> stateless
# 즉, 모델에게 어떤 말을 건네면 모델은 답을 한 직후에 대화 내용을 까먹게 됨 -> 메모리가 없으니까 내용을 저장하지 않음

# 챗GPT에는 메모리가 탑재되있기 때문에 실제로 어떤 사람과 얘기하고 있다는 느낌을 들게 함
# 챗봇이 이전의 대화 내용이나 질문을 기억하고 답할 수 있으니까

# 즉, LLM 모델 자체에는 메모리를 지원하지 않음

# 메모리 종류와는 무관하게 API는 다 똑같음
# 그래서 모든 메모리는 return_messages 변수, save_context, load_memory_variables 등 이라는 함수를 갖고 있음

In [3]:
# 4. Conversation Summary Buffer Memory
# ConversationSummaryMemory + ConversationBufferMemory
# 우선, 메모리에 보내온 메시지의 수를 저장함
# 그리고 limit에 다다른 순간, 그동안 무슨 일이 일어났는지 잊어버리는 것 대신 오래된 메시지들을 요약함
# 즉, 가장 최근의 메시지와 가장 오래 전에 주고 받은 메시지 모두 잊혀지지 않고 요약되어 가지고 있다는 거

In [6]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,  # 메모리를 실행하는데 비용이 듬 -> 요약하는데 llm을 사용하고 있기 때문에
    max_token_limit=150,  # 요약하지 않고 가지고 있을 최근 메모리의 윈도우
    return_messages=True,
)

In [7]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

In [8]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [9]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [10]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [11]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [12]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [14]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [15]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [18]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
add_message("What color do you like?", "I like red!")

In [19]:
get_history()  # 과거 대화가 요약되어 저장됨과 동시에 최근에 입력된 대화내용은 그대로 저장되어 있음

{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI expresses excitement and wishes it could visit South Korea. Nicolas asks how far Korea is from Argentina.'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='What color do you like?'),
  AIMessage(content='I like red!')]}